In [ ]:
import numpy as np
from plots import *
import scipy.io
import pandas as pd
from pathlib import Path

def plot_mags():
    m = get_mags(wf, sr=44100, t_win=1, num_wins=30, dict=True)
    mags = m['mags']
    freq_ax = m['freq_ax']
    plt.plot(freq_ax, np.log10(mags)*20)
    plt.title(fn)
    plt.show()
    
# get the main directory in my computer
main_path_str = "C:\\Users\\Owner\OneDrive\\Desktop\\SOAE Data\\"
# we'll process each subfolder separately since each is likely to have its own quirks

In [ ]:
subfolder = "Curated Data"
    
# We'll build our dataframe by making a dictionary of lists and appending to them
data = {
    'filepath': [],
    'wf': [],  
    'species': [],
    'sr': [],
}

# First navigate to our directory
directory_path = Path(main_path_str + subfolder)

# track which file we're on
n_files = sum(1 for _ in directory_path.rglob('*') if _.is_file())
i=0

# now loop through all files in that collection
for fp in directory_path.rglob('*'):
    # Check if it's a file
    if fp.is_file():  
        i += 1
        print(f"Processing file {i}/{n_files}")
        # Cut off the beginning of the filepath since it's unnecessary for our dataframe (fps = file path shortened)
        main_path = Path(main_path_str)
        fps = str(fp.relative_to(main_path))
        
        # Get the filename itself (without its containing folders)
        fn = fp.name
        # now we actually open the waveform here
        # Check if it's a .txt or .mat file
        try:
            if fp.suffix == '.mat':
                mat = scipy.io.loadmat(fp)
                if 'wf' in mat:
                    wf = np.squeeze(mat['wf'])
                else: 
                    print(f"Not sure how to process {fp}")
            if fp.suffix == '.txt':
                wf = np.loadtxt(fp)
            # Let's make sure this waveform is a 1D array
            if len(wf.shape) > 1:
                if fn == 'cricket_177.txt':
                    wf = wf[:, 1]
                else:
                    print(f"Waveform from {fps} isn't 1D!")
        except:
            f"Uh oh! Issue when loading {fp}"
            
        # if str(fps).split("\\")[1]=='Tree Cricket':
        #     plot_mags()
        
        
        # try and get the species name
        fn_species = fn.split("_")[0]
        
        match fn_species:
            case 'anole':
                species = "Anole"
            case 'cricket':
                species = "Cricket"
            case 'human':
                species = "Human"
            case 'owl':
                species = "Owl"
            case _:
                species = ""
        
        if species != "Owl":
            sr = 44100
        else:
            sr = 0
            
                
        # add everything to our df dict
        data['filepath'].append(fps)
        data['wf'].append(wf)
        data['species'].append(species)
        data['sr'].append(sr)

# turn this into a pandas dataframe
df = pd.DataFrame(data)
df['filepath'] = df['filepath'].astype(str)
df['species'] = df['species'].astype(str)
# save this as a parquet file for efficient dataframe storage (use pyarrow since the 'wf' column has different length lists)
df.to_parquet(f'{subfolder}.parquet', engine='pyarrow')

In [ ]:
subfolder = "Curated Data"
    
# We'll build our dataframe by making a dictionary of lists and appending to them
data = {
    'filepath': [],
    'wf': [],  
    'species': [],
    'sr': [],
}

# First navigate to our directory
directory_path = Path(main_path_str + subfolder)

# track which file we're on
n_files = sum(1 for _ in directory_path.rglob('*') if _.is_file())
i=0

# now loop through all files in that collection
for fp in directory_path.rglob('*'):
    # Check if it's a file
    if fp.is_file():  
        i += 1
        print(f"Processing file {i}/{n_files}")
        # Cut off the beginning of the filepath since it's unnecessary for our dataframe (fps = file path shortened)
        main_path = Path(main_path_str)
        fps = str(fp.relative_to(main_path))
        
        # Get the filename itself (without its containing folders)
        fn = fp.name
        # now we actually open the waveform here
        # Check if it's a .txt or .mat file
        try:
            if fp.suffix == '.mat':
                mat = scipy.io.loadmat(fp)
                if 'wf' in mat:
                    wf = np.squeeze(mat['wf'])
                else: 
                    print(f"Not sure how to process {fp}")
            if fp.suffix == '.txt':
                wf = np.loadtxt(fp)
            # Let's make sure this waveform is a 1D array
            if len(wf.shape) > 1:
                if fn == 'cricket_177.txt':
                    wf = wf[:, 1]
                else:
                    print(f"Waveform from {fps} isn't 1D!")
        except:
            f"Uh oh! Issue when loading {fp}"
            
        # if str(fps).split("\\")[1]=='Tree Cricket':
        #     plot_mags()
        
        
        # try and get the species name
        fn_species = fn.split("_")[0]
        
        match fn_species:
            case 'anole':
                species = "Anole"
            case 'cricket':
                species = "Cricket"
            case 'human':
                species = "Human"
            case 'owl':
                species = "Owl"
            case _:
                species = ""
        
        if species != "Owl":
            sr = 44100
        else:
            sr = 0
            
                
        # add everything to our df dict
        data['filepath'].append(fps)
        data['wf'].append(wf)
        data['species'].append(species)
        data['sr'].append(sr)

# turn this into a pandas dataframe
df = pd.DataFrame(data)
df['filepath'] = df['filepath'].astype(str)
df['species'] = df['species'].astype(str)
# save this as a parquet file for efficient dataframe storage (use pyarrow since the 'wf' column has different length lists)
df.to_parquet('curated_data.parquet', engine='pyarrow')

In [ ]:
subfolder = "Extra Owl"
    
# We'll build our dataframe by making a dictionary of lists and appending to them
data = {
    'filepath': [],
    'wf': [],  
    'species': [],
    'sr': [],
}

species = "Owl"

# First navigate to our directory
directory_path = Path(main_path_str + subfolder)

# track which file we're on
n_files = sum(1 for _ in directory_path.rglob('*') if _.is_file())
i=0

# now loop through all files in that collection
for fp in directory_path.rglob('*'):
    # Check if it's a file
    if fp.is_file():  
        i += 1
        print(f"Processing file {i}/{n_files}")
        # Cut off the beginning of the filepath since it's unnecessary for our dataframe (fps = file path shortened)
        main_path = Path(main_path_str)
        fps = str(fp.relative_to(main_path))
        
        # Get the filename itself (without its containing folders)
        fn = fp.name
        # now we actually open the waveform here
        # Check if it's a .txt or .mat file
        try:
            if fp.suffix == '.mat':
                mat = scipy.io.loadmat(fp)
                if 'wf' in mat:
                    wf = np.squeeze(mat['wf'])
                else: 
                    print(f"Not sure how to process {fp}")
            if fp.suffix == '.txt':
                wf = np.loadtxt(fp)
            # Let's make sure this waveform is a 1D array
            if len(wf.shape) > 1:
                print(f"Waveform from {fps} isn't 1D!")
        except:
            f"Uh oh! Issue when loading {fp}"
            
        if str(fps).split("\\")[1]=='Oldenberg Data (2013) (44.1kHz)':
            sr = 44100
        elif str(fps).split("\\")[1]=='Pim owl files (48 kHz)':
            sr = 48000
        else:
            print("UH OH WHERE ARE WE")
        
            
                
        # add everything to our df dict
        data['filepath'].append(fps)
        data['wf'].append(wf)
        data['species'].append(species)
        data['sr'].append(sr)

# turn this into a pandas dataframe
df = pd.DataFrame(data)
df['filepath'] = df['filepath'].astype(str)
df['species'] = df['species'].astype(str)
# save this as a parquet file for efficient dataframe storage (use pyarrow since the 'wf' column has different length lists)
df.to_parquet(f'{subfolder}.parquet', engine='pyarrow')